In [146]:
# libraries
import numpy as np
import pandas as pd
import altair as alt
from sklearn.decomposition import PCA

# warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# display settings
pd.options.display.max_columns = None

# PSTAT 100 Project plan

## Group information

**Group members**:

**Contributions**:
1. Member 1 studied the data documentation and prepared the data description.
2. Member 2 worked on tidying the dataset.
3. Member 3 worked on variable summaries and explanatory plots
4. Member 4 worked on 

---
## 0. Background

California has a well-developed public transportation system that includes buses, trains, light rail, and subways. The state's largest public transportation agency is the Los Angeles County Metropolitan Transportation Authority (Metro), which serves the Los Angeles metropolitan area with an extensive network of bus and rail lines. Other major public transportation agencies in California include the San Francisco Municipal Transportation Agency (SFMTA) and the San Diego Metropolitan Transit System (MTS). In addition to these larger agencies, many smaller cities and towns throughout California have their own public transportation systems, which may include buses, shuttles, or other types of services. These systems often provide connections to regional or statewide transportation networks, making it possible for people to travel throughout the state using public transportation. While many areas have well-established public transportation systems, there may be gaps or limitations in service in some parts of the state, particularly in more rural or remote areas.

The Walkable Distance to Public Transit dataset, available on data.ca.gov, provides information on the number of households in California that are within a certain distance of public transit stops. The data is based on estimates of walking distance from households to the nearest transit stop, as well as information on the types of transit services available at each stop. 

The dataset is organized by county, and includes information on the number and percentage of households within various walking distance ranges of transit stops. For example, the data might indicate how many households are within a 5-minute walk of a transit stop, as well as how many households are within a 10-minute or 15-minute walk.

The dataset also provides information on the types of transit services available at each stop, including whether the stop serves buses, trains, or other modes of transportation. This information can be useful for understanding the accessibility of public transportation in different areas of the state, and for identifying opportunities to improve transit service or expand transit infrastructure.

Overall, the Walkable Distance to Public Transit dataset is a valuable resource for policymakers, transportation planners, and researchers who are interested in understanding the availability and accessibility of public transportation in California.

---
## 1. Data description

### Basic Information

The data is the percent of the population that resides within 1/2 mile of a major transit location (bus/rail/ferry stop) in four California regions, and whose waiting time is less than 15 minutes during peak commute hours. The data is stratified by 8 race/ethnicity groups and includes both geographic information and statistical reliability measurements.

The data was collected by the California Department of Public Health, as part of the "Health Communities Data and Indicators Project (HCI). The goal of the project was to evaluate how city plans and policies affect community health. The data includes 2012 Transit Stops from the San Diego and Southern California Association of Governments, as well as the Metropolitan Transportation Commission. It also includes 2008 Transit Stops from the Sacramento Council of Government and 2010 block-level population data from the U.S. Census Bureau. The data is updated decennially. The four California regions are defined as the following:
* Southern California (SCAG): Imperial, Los Angeles, Orange, Riverside, San Bernardino, and Ventura 
* Sacramento (SACOG): Placer, Sacramento, and Yolo 
* Bay Area (MTC): Contra Costa, Marin, Napa, San Francisco, San Mateo, Santa Clara, Solano, Sonoma 
* San Diego County

Data values were obtained using automated methods to download information from various public websites. One important data set was from the 2010-2012 California Household Travel Survey. Multiple data collection methods were used in this survey, including computer-assisted telephone interviews and online/mail surveys. To identify census blocks inside 1/2 mile of the transit stops, geospatial software was used. In order to compile the data into one data set, the census blocks from the 2010 U.S. Census were merged with the blocks from the transit data, and population counts were aggregated by census tract, cities/towns, county, and region. The data was processed into Excel files with standard formats. 

The population is adults aged 18 years and over, who reside in the four California regions. The sampling frame includes adults in these four regions, with access to telephone or mail services. The sampling mechanism for the respective year (2008 or 2012) is a probability sample because the surveys downloaded by the HCI project were sent to randomly selected adults However, the scope of inference has limitations. The data is from the year 2012 for the SCAG, MTC, and San Diego regions; while, 2008 for the SACOG region. Some transit stops and services may have changed during that time period. As well, the population data was collected from the 2010 U.S. Census, which is a different time period than the transit data (2008, 2012). Therefore, some variation may exist if demographics changed.

### Data semantics and structure

Name | Variable description | Type | Units of measurement
---|---|---|---
year | year when data was reported | Numeric | Calendar year 
race_eth_name | name of the different races/ethnicties ('AfricanAm', 'AIAN', 'Asian', 'Latino', 'Multiple', 'NHOPI', 'Other', 'Total', 'White')| Object | Name
geotype | describes the level of geography for data in that row ('RE'=region, 'CT'=census tract, 'PL'=place/town/city, 'CO'=county)  | Object | Name
geoname | name of the city/town | Object | Name 
county_name | name of the county | Object | Name
region_name | name of the region ('Sacramento Area', 'Bay Area', 'San Diego', 'Southern California') | Object | Name
pop_trans_acc | number of residents that live within 1/2 mile of public transportation | Numeric | Integer
pop2010 | total number of residents that reside in that county | Numeric | Integer
p_trans_acc | the percent of residents that live within 1/2 mile of public transportation | Numeric | Float
LL_95CI | lower limit of the 95th confidence interval for p_trans_acc | Numeric | Float
UL_95CI | upper limit of the 95th confidence interval for p_trans_acc | Numeric| Float
se | standard errror | Numeric| Float
rse | relative standard error | Numeric| Float

### Data Overview

In [147]:
# load tidied data and print rows
data = pd.read_csv(
    'tidy-data', 
    dtype = {'pop_trans_acc':'Int64',
             'county_fips': 'Int64'},
    index_col = 0
)

data.head()

,year,race_eth_code,race_eth_name,geotype,geotypevalue,geoname,county_name,county_fips,region_name,region_code,pop_trans_acc,pop2010,p_trans_acc,LL_95CI,UL_95CI,se,rse
0,2008,3,AfricanAm,CO,6061,Placer,Placer,6061,Sacramento Area,8,55,4427,0.012424,0.009161,0.015687,0.001665,13.399974
1,2008,1,AIAN,CO,6061,Placer,Placer,6061,Sacramento Area,8,51,2080,0.024519,0.017873,0.031166,0.003391,13.830066
2,2008,2,Asian,CO,6061,Placer,Placer,6061,Sacramento Area,8,117,19963,0.005861,0.004802,0.006920,0.000540,9.217872
3,2008,4,Latino,CO,6061,Placer,Placer,6061,Sacramento Area,8,1835,44710,0.041042,0.039203,0.042881,0.000938,2.286029
4,2008,7,Multiple,CO,6061,Placer,Placer,6061,Sacramento Area,8,241,10658,0.022612,0.019790,0.025435,0.001440,6.368321


---
# 2. Initial Explorations

### Basic properties of the dataset

##### (a) Dimensions of the data

In [148]:
data.shape

(66006, 17)

The data set contains 66,006 rows (66,005 observations / 1 header) and 17 columns (variables).

##### (b) Missing values

In [149]:
missing_values = data.isna().sum()
missing_values 

year                 0
race_eth_code        0
race_eth_name        0
geotype              0
geotypevalue         0
geoname             54
county_name         63
county_fips         63
region_name          0
region_code          0
pop_trans_acc     1182
pop2010              0
p_trans_acc       1557
LL_95CI           1182
UL_95CI           1557
se                1182
rse              14700
dtype: int64

Yes, there are missing values in the data set. The variables missing are 'geoname,' 'county_name,' 'county_fips,' 'pop_trans_acc,' 'p_trans_acc' and the statsitical reliability measurements: ('LL_95CI,' 'UL_95CI,' 'se,' and 'rse.') The majority of variables missing are from these statistical measurements. In addition, none of the variables are missing more than 2.5% of the time, except 'rse' (22.3%). 

The variables are missing becasue...

##### (c) Variable summaries

**_Value counts of the race/ethnicity groups by year_**

In [150]:
data.groupby(['year']).race_eth_name.value_counts()

year  race_eth_name
2008  AIAN              518
      AfricanAm         518
      Asian             518
      Latino            518
      Multiple          518
      NHOPI             518
      Other             518
      Total             518
      White             518
2012  AIAN             6816
      AfricanAm        6816
      Asian            6816
      Latino           6816
      Multiple         6816
      NHOPI            6816
      Other            6816
      Total            6816
      White            6816
Name: race_eth_name, dtype: int64

* Since the 2008 study was confined to the Sacramento region, there are less observations for each race/ethnicity group. In the year 2012, the study incorporated three regions (Southern California, Bay Area, San Diego) and therefore, had more observations for each group. 

**_Summary statistics of the percent of residents that reside within 1/2 mile of public transportation. Sorted by year, region, and race/ethnicity_**

In [151]:
data.groupby(['year', 'region_name', 'race_eth_name']).p_trans_acc.describe()

count      mean       std  min  25%  \
year region_name         race_eth_name                                         
2008 Sacramento Area     AIAN            512.0  0.172234  0.324438  0.0  0.0   
                         AfricanAm       510.0  0.176581  0.325833  0.0  0.0   
                         Asian           515.0  0.170510  0.321131  0.0  0.0   
                         Latino          516.0  0.171872  0.317324  0.0  0.0   
                         Multiple        515.0  0.172559  0.318749  0.0  0.0   
                         NHOPI           492.0  0.178389  0.333260  0.0  0.0   
                         Other           481.0  0.171500  0.333844  0.0  0.0   
                         Total           517.0  0.169488  0.314589  0.0  0.0   
                         White           517.0  0.169520  0.315144  0.0  0.0   
2012 Bay Area            AIAN           1753.0  0.536266  0.440682  0.0  0.0   
                         AfricanAm      1801.0  0.538805  0.428979  0.0  0.0   
                         Asian          1806.0  0.526217  0.426534  0.0  0.0   
                         Latino         1810.0  0.535067  0.425425  0.0  0.0   
                         Multiple       1809.0  0.522851  0.425928  0.0  0.0   
                         NHOPI          1738.0  0.543604  0.437997  0.0  0.0   
                         Other          1744.0  0.536302  0.440422  0.0  0.0   
                         Total          1811.0  0.524348  0.424028  0.0  0.0   
                         White          1811.0  0.516707  0.424981  0.0  0.0   
     San Diego           AIAN            672.0  0.355468  0.441479  0.0  0.0   
                         AfricanAm       678.0  0.365791  0.438789  0.0  0.0   
                         Asian           679.0  0.364138  0.435608  0.0  0.0   
                         Latino          680.0  0.370478  0.437872  0.0  0.0   
                         Multiple        679.0  0.360166  0.433166  0.0  0.0   
                         NHOPI           665.0  0.359390  0.441381  0.0  0.0   
                         Other           652.0  0.360470  0.448037  0.0  0.0   
                         Total           681.0  0.363501  0.433842  0.0  0.0   
                         White           681.0  0.355831  0.430874  0.0  0.0   
     Southern California AIAN           4149.0  0.357655  0.451311  0.0  0.0   
                         AfricanAm      4286.0  0.363317  0.445586  0.0  0.0   
                         Asian          4279.0  0.359181  0.441773  0.0  0.0   
                         Latino         4297.0  0.364241  0.442027  0.0  0.0   
                         Multiple       4276.0  0.360061  0.442245  0.0  0.0   
                         NHOPI          3727.0  0.332688  0.447287  0.0  0.0   
                         Other          4111.0  0.363264  0.452759  0.0  0.0   
                         Total          4300.0  0.360776  0.439914  0.0  0.0   
                         White          4299.0  0.357181  0.439186  0.0  0.0   

                                             50%       75%  max  
year region_name         race_eth_name                           
2008 Sacramento Area     AIAN           0.000000  0.167342  1.0  
                         AfricanAm      0.000000  0.200796  1.0  
                         Asian          0.000000  0.114129  1.0  
                         Latino         0.000000  0.190291  1.0  
                         Multiple       0.000000  0.189566  1.0  
                         NHOPI          0.000000  0.166667  1.0  
                         Other          0.000000  0.055556  1.0  
                         Total          0.000000  0.169238  1.0  
                         White          0.000000  0.171211  1.0  
2012 Bay Area            AIAN           0.634146  1.000000  1.0  
                         AfricanAm      0.638171  1.000000  1.0  
                         Asian          0.583761  1.000000  1.0  
                         Latino         0.613572  1.000000  1.0  
          

* This chart shows how access to public transportation varies across each region. Across all race/ethnicity groups, the Bay Area has the highest mean access to public transportation. The Sacramento Area has the lowest mean access. There are similar public transportation rates between the San Diego and Southern California regions. 

* Two statistics that stands out are the 50th and 75th percentile, for public transportation rates. The Bay Area is the only region where the 50th percentile is not 0.00. Additionally, the 75th percentile for the Bay Area is 1.00 across all race/ethnicity groups, while it is less than 1.00 for the other regions.

**_Value counts of the counties sorted by year and region_**

In [152]:
data.groupby(['year', 'region_name']).county_name.value_counts()

year  region_name          county_name   
2008  Sacramento Area      Sacramento         3177
                           Placer              954
                           Yolo                495
2012  Bay Area             Santa Clara        3573
                           Alameda            3438
                           Contra Costa       2358
                           San Francisco      1791
                           San Mateo          1719
                           Sonoma             1242
                           Solano              972
                           Marin               792
                           Napa                468
      San Diego            San Diego          6138
      Southern California  Los Angeles       22392
                           Orange             5625
                           Riverside          4797
                           San Bernardino     3789
                           Ventura            1782
                           Imperial     

* This chart lists the counties in each of the four regions and shows the number of observations per each county. It is important to note that the San Diego region only has one county named 'San Diego'. Out of the counties, Los Angeles had the most observations at 22,392 almost 4x higher than the next highest county of San Diego.

**_Value counts of the level of geography sorted by year and region_**

In [153]:
data.groupby(['year', 'region_name']).geotype.value_counts()

year  region_name          geotype
2008  Sacramento Area      CT          3987
                           PL           639
                           CO            27
                           RE             9
2012  Bay Area             CT         14292
                           PL          1989
                           CO            81
                           RE             9
      San Diego            CT          5652
                           PL           477
                           CO             9
      Southern California  CT         35604
                           PL          3168
                           CO            54
                           RE             9
Name: geotype, dtype: int64

* 'geotype' describes the level of geography for data in that row _('RE'=region, 'CT'=census tract, 'PL'=place/town/city, 'CO'=county)_. Having more values for 'PL' means we have more granular data, as we can observe public transportation rates in a specific city. One example where the geotype is PL is from row 4015: 'Auburn city (PL) / Placer (CO) / Sacramento Area (RE). This allows us to examine a specific city, compared to the geotype being 'CO' or 'RE,' where we could only examine a county or region.

### Exploratory analysis

**(a) Access to public transportation by region and race/ethnicity**

In [156]:
alt.data_transformers.enable('json')
alt.Chart(data).mark_bar().encode(
    x = alt.X('race_eth_name', title = ''), 
    y = alt.Y('mean(p_trans_acc)', 
              scale=alt.Scale(domain=[0, 0.65]),
              title = 'Mean Percentage of Access to Public Transportation')
).properties(
    width = 200,
    height = 325
).facet(
    column = 'region_name'
)

alt.FacetChart(...)

The bar charts compare the mean percentage of residents that live within 1/2 mile of public transportation across different race/ethnicity groups. It also separates each bar chart accoding to the different region and shows that region is a more significant indicator in access to public transportation, compared to race/ethnicity; however, differences are still present between each race/ethnicity.

**(b) Population's relationship with access to public transportation**

In [155]:
# alt.data_transformers.enable('json')
alt.Chart(
    data[(data.geotype == 'CO')]
).mark_point().encode(
    x = alt.X('pop2010', 
              scale = alt.Scale(zero = False, type = 'pow', exponent = 0.25),
              title = '2010 County Population'),
    y = alt.Y('mean(p_trans_acc)', 
              title = 'Mean Percentage of Access to Public Transportation')
).properties(
    width = 300,
    height = 300
).facet(
    column = 'region_name'
)

alt.FacetChart(...)

The scatter plots shows how access to public transportation varies with the population in each **county** in the region (the points represent each county). Counties with larger populations, specifically Southern California, have higher access to public transportation, but this relationship is less apparent in the other regions. 

---
## 3. Planned work

### Questions

Two focused questions that we plan to explore.

1. Has access to public transportation improved from 2008 to 2012?
2. Does access to public transportation significantly differ across race / ethnicity groups?

### Proposed approaches

How we might approach the question.

1. *Approach 1 here* 
2. *Approach 2 here*

---
## Submission Checklist
1. Save file to confirm all changes are on disk
2. Run *Kernel > Restart & Run All* to execute all code from top to bottom
3. Save file again to write any new output to disk
4. Generate PDF and submit to Gradescope